In [1]:
import meshio
import numpy as np
from multiprocessing import Pool
import time
import functools
from queue import PriorityQueue

#from ugs import *
# Need to make sure it's installed: pip3 install line_profiler
%load_ext line_profiler

In [2]:
@functools.lru_cache(maxsize=32768)
def distance(current, _next):
    # from https://stackoverflow.com/a/1401828
    if current == _next:
        return 0
    return int(np.linalg.norm(mesh.points[current]-mesh.points[_next]))

@functools.lru_cache(maxsize=183746)
def graph_neighbours(current):
    # Get the points from the cells which have the same point in them
    points = np.unique(mesh.cells['triangle'][np.where(mesh.cells['triangle']==current)[0]])
    # remove the current point from these results:
    points = points[points != current]
    
    # Get the elevation of those connected points
    elevations = mesh.point_data['Z'][points]
    # Return a list of connected points, as long as they are above sea-level
    return points[elevations >= 0]
    

def cost_search(start, travel_cost_function, max_distance=100000, graph_neighbours=graph_neighbours):
    frontier = PriorityQueue()  # The priority queue means that we can find the least cost path to continue
    frontier.put(start, 0)      # from, along any path, meaning the resulting paths should always be the least-cost
                                # path to get to that point.
    came_from = {}
    cost_so_far = {}
    dist_so_far = {}
    came_from[start] = None
    cost_so_far[start] = 0
    dist_so_far[start] = 0
    
    while not frontier.empty():
        current = frontier.get()
        for _next in graph_neighbours(current):
            # Calculate the cost of going to this new point.
            new_cost = cost_so_far[current] + travel_cost_function(current, _next)
            # Calculate the eulerian distance to this new point.
            new_dist = dist_so_far[current] + distance(current, _next)

            # The max_distance check tells the algorithm to stop once we start getting too far away from the starting point.
            if (_next not in cost_so_far or new_cost < cost_so_far[_next]) and new_dist < max_distance:
                cost_so_far[_next] = new_cost
                dist_so_far[_next] = new_dist
                priority = new_cost
                frontier.put(_next, priority)
                came_from[_next] = current

        
    return came_from, cost_so_far

def cost_search(start, travel_cost_function, max_fuel=1000, graph_neighbours=graph_neighbours):
    frontier = PriorityQueue()  # The priority queue means that we can find the least cost path to continue
    frontier.put(start, 0)      # from, along any path, meaning the resulting paths should always be the least-cost
                                # path to get to that point.
    came_from = {}
    cost_so_far = {}
    dist_so_far = {}
    came_from[start] = None
    cost_so_far[start] = 0
    dist_so_far[start] = 0
    
    while not frontier.empty():
        current = frontier.get()
        for _next in graph_neighbours(current):
            # Calculate the cost of going to this new point.
            new_cost = cost_so_far[current] + travel_cost_function(current, _next)
            # Calculate the eulerian distance to this new point.
            new_dist = dist_so_far[current] + distance(current, _next)

            # The max_distance check tells the algorithm to stop once we start getting too far away from the starting point.
            if (_next not in cost_so_far or new_cost < cost_so_far[_next]) and new_cost <= max_fuel:
                cost_so_far[_next] = new_cost
                dist_so_far[_next] = new_dist
                priority = new_cost
                frontier.put(_next, priority)
                came_from[_next] = current

        
    return came_from, cost_so_far, dist_so_far

def get_total_cost_for_point(start, travel_cost_function, max_distance=100000, graph_neighbours=graph_neighbours):
    came_from, cost_so_far, dist_so_far = cost_search(start, travel_cost_function, max_distance, graph_neighbours=graph_neighbours)
    
    # Find the edge nodes, and add up their costs to get the total
    total_cost = 0
    for k in came_from.keys():             # For all the points we've visited,
        if k not in came_from.values():    # Find all the points that haven't been 'came_from'
            total_cost += cost_so_far[k]
            
    return total_cost

def get_total_distance_for_all_paths_to_point(start, travel_cost_function, max_fuel=1000, graph_neighbours=graph_neighbours):
    came_from, cost_so_far, dist_so_far = cost_search(start, travel_cost_function, max_fuel, graph_neighbours=graph_neighbours)
    
    # Find the edge nodes, and add up their costs to get the total
    total_dist = 0
    for k in came_from.keys():             # For all the points we've visited,
        if k not in came_from.values():    # Find all the points that haven't been 'came_from'
            total_dist += dist_so_far[k]
            
    return total_dist

def get_from_cell(cell, travel_cost_function, max_distance=100000):
    start = cell[1]
    return get_from_point(start, travel_cost_function, max_distance)

def get_dist_from_point(point, travel_cost_function, max_fuel=1000, graph_neighbours=graph_neighbours):
    # Return a tuple of (the point id, it's cost)
    total_dist = get_total_distance_for_all_paths_to_point(point, travel_cost_function, max_fuel, graph_neighbours=graph_neighbours)
    #print(os.getpid(), distance.cache_info())
    return (point, total_dist)

def get_from_point(point, travel_cost_function, max_distance=100000, graph_neighbours=graph_neighbours):
    # Return a tuple of (the point id, it's cost)
    total_cost = get_total_cost_for_point(point, travel_cost_function, max_distance, graph_neighbours)
    #print(os.getpid(), distance.cache_info())
    return (point, total_cost)

In [3]:
# We need to define a way to calculate cost
@functools.lru_cache(maxsize=1048576)
def distance_with_elevation_scaled(current, _next):
    # The travel_cost can be any function, including just the distance.
    # Here, we exagerate the elevation difference, to make changing elevation more costly
    if current == _next:
        return 0
    
    z_scaling = 100.  # 100. is a random number to pick, but has quite a big impact on the resulting paths.
    
    new_current = np.append(mesh.points[current][:2], mesh.point_data['Z'][current] * z_scaling)
    new_next    = np.append(mesh.points[_next][:2],   mesh.point_data['Z'][_next]   * z_scaling)
    
    return int(np.linalg.norm(new_current - new_next))  # return as Int, just for niceness

In [4]:
# We need to define a way to calculate cost
@functools.lru_cache(maxsize=1048576)
def elevation_only(current, _next):
    # Only take into account elevation changes for costs
    if current == _next:
        return 0
    return int(abs(mesh.point_data['Z'][current] - mesh.point_data['Z'][_next]) + distance(current, _next)*0.004)

In [5]:
infile = "earth/data/globe.vtk"
outfile = "costed_globe_new.vtk"
mesh = meshio.read(infile)

max_fuel = 2500
                      # A smaller value means visiting far fewer nodes, so it speeds things up a lot

In [6]:
# Choose which cost function you want
travel_cost = elevation_only

## Prepare input data

We don't want to calculate the LEC of a point below sea-level, so here we find all the points above sea-level, so they can be used as starting points.

In [7]:
points_above_sealevel = np.nonzero(mesh.point_data['Z'] >= 0)[0][::-1]

In [8]:
print("Total starting points available: ", points_above_sealevel.shape[0])

Total starting points available:  183646


## Prepare for parallel execution

We want to use a parallel map function to calculate the cost for all points. We can make this easier by 'baking in' the parameters of a function we know, and leave only the `starting point` as a variable.

In [9]:
from functools import partial

# We bake in the mesh and travel_cost function into the get_from_point function
get_from_point_in_mesh = partial(get_dist_from_point, travel_cost_function = travel_cost, max_fuel = max_fuel)

In [10]:
#%lprun -f get_from_point_in_mesh get_from_point_in_mesh(points_above_sealevel[100])

In [11]:
# Now we can use the get_from_point function with only a point ID as a parameter, by going via the new get_from_point_in_mesh function
# Here we do a test, and see the output format: (point, total cost of all paths to that point)
print(travel_cost(points_above_sealevel[100], points_above_sealevel[101]))
print(get_from_point_in_mesh(points_above_sealevel[100]))
print(graph_neighbours.cache_info())

26021
(651092, 24994625)
CacheInfo(hits=580, misses=231, maxsize=183746, currsize=231)


In [12]:
# Output data variables
all_costs = []
mesh.point_data['cost'] = np.zeros_like(mesh.point_data['Z'])

p = Pool(6)
start = 0
inc = 1000
stop = inc

while start < points_above_sealevel.shape[0]:
    start_time = time.time()
    # Here we use a parallel map function to send out the chunk across the CPUs in the Pool
    costs = p.map(get_from_point_in_mesh, points_above_sealevel[start:stop])
    print("From ", start, " to ", stop, " took ", time.time() - start_time, "seconds, starting with point", points_above_sealevel[start], ". Percent complete: ", 100*(float(stop)/points_above_sealevel.shape[0]))
    # Save the data
    all_costs.extend(costs)
    
    # Write out data progressively, so we can see progress in Paraview
    for i in all_costs:
        mesh.point_data['cost'][i[0]] = i[1]
    meshio.write(outfile, mesh)
    
    # move to the next chunk of data
    start += inc
    stop += inc
    if stop >= points_above_sealevel.shape[0]:
        stop = points_above_sealevel.shape[0]-1

From  0  to  1000  took  258.11433601379395 seconds, starting with point 655344 . Percent complete:  0.5445258813151389
From  1000  to  2000  took  143.36398315429688 seconds, starting with point 644591 . Percent complete:  1.0890517626302778
From  2000  to  3000  took  141.87588477134705 seconds, starting with point 637313 . Percent complete:  1.6335776439454168
From  3000  to  4000  took  156.11566472053528 seconds, starting with point 633836 . Percent complete:  2.1781035252605556
From  4000  to  5000  took  103.78504371643066 seconds, starting with point 629731 . Percent complete:  2.722629406575695
From  5000  to  6000  took  125.40420818328857 seconds, starting with point 628123 . Percent complete:  3.2671552878908336
From  6000  to  7000  took  86.1420795917511 seconds, starting with point 624947 . Percent complete:  3.8116811692059724
From  7000  to  8000  took  139.74660301208496 seconds, starting with point 623947 . Percent complete:  4.356207050521111
From  8000  to  9000  t

From  67000  to  68000  took  19.643401861190796 seconds, starting with point 413537 . Percent complete:  37.02775992942944
From  68000  to  69000  took  23.94182777404785 seconds, starting with point 410172 . Percent complete:  37.57228581074458
From  69000  to  70000  took  16.26025152206421 seconds, starting with point 407852 . Percent complete:  38.11681169205972
From  70000  to  71000  took  20.97862958908081 seconds, starting with point 403196 . Percent complete:  38.66133757337486
From  71000  to  72000  took  20.476336002349854 seconds, starting with point 399945 . Percent complete:  39.20586345469
From  72000  to  73000  took  17.16881036758423 seconds, starting with point 397068 . Percent complete:  39.75038933600514
From  73000  to  74000  took  16.521696090698242 seconds, starting with point 392861 . Percent complete:  40.29491521732028
From  74000  to  75000  took  20.61347460746765 seconds, starting with point 389402 . Percent complete:  40.83944109863542
From  75000  to 

From  133000  to  134000  took  16.145644903182983 seconds, starting with point 180766 . Percent complete:  72.96646809622861
From  134000  to  135000  took  16.43948221206665 seconds, starting with point 177063 . Percent complete:  73.51099397754375
From  135000  to  136000  took  17.9876229763031 seconds, starting with point 173452 . Percent complete:  74.05551985885889
From  136000  to  137000  took  17.112582445144653 seconds, starting with point 169529 . Percent complete:  74.60004574017402
From  137000  to  138000  took  17.45764183998108 seconds, starting with point 166489 . Percent complete:  75.14457162148916
From  138000  to  139000  took  13.203635215759277 seconds, starting with point 160869 . Percent complete:  75.6890975028043
From  139000  to  140000  took  12.802222728729248 seconds, starting with point 154277 . Percent complete:  76.23362338411944
From  140000  to  141000  took  14.409239292144775 seconds, starting with point 151527 . Percent complete:  76.778149265434

## Output

Show some of the results:

In [13]:
print all_costs[:100]

SyntaxError: Missing parentheses in call to 'print' (<ipython-input-13-5d2897a31d53>, line 1)